# Resume Tailor Agent

An intelligent agent that tailors your LaTeX resume to specific job postings while preserving formatting and maintaining accuracy.

## Features

- **LaTeX-Safe**: Preserves LaTeX formatting and syntax
- **Iterative**: Supports multiple revision rounds
- **Job-Focused**: Analyzes job postings and matches requirements
- **ATS-Optimized**: Uses keywords naturally for applicant tracking systems
- **Validation**: Checks LaTeX syntax before output

---

## Setup

Import required libraries and configure the environment.

## API Provider Configuration

This notebook supports multiple AI providers. Configure your credentials in the `.env` file:

### Option 1: OpenAI (Recommended for getting started)
```bash
OPENAI_API_KEY=sk-your-openai-key-here
```

### Option 2: AWS Bedrock (Production-ready)
```bash
# Using long-term API key (recommended)
AWS_BEARER_TOKEN_BEDROCK=your-long-term-bedrock-key
AWS_REGION=us-east-1

# OR using standard AWS credentials
AWS_ACCESS_KEY_ID=your-access-key
AWS_SECRET_ACCESS_KEY=your-secret-key
AWS_REGION=us-east-1
```

The notebook will automatically detect which credentials are available and use them.

---

In [ ]:
# Core imports
import os
from pathlib import Path
from dotenv import load_dotenv

# Strands SDK
from strands import Agent, tool

# Utilities
import json
from datetime import datetime

# Load environment variables
load_dotenv()

print("✅ Imports successful!")
print(f"Python Path: {Path.cwd()}")

## Configuration

Set up paths and verify environment.

In [ ]:
# Project paths
PROJECT_ROOT = Path.cwd()
PROMPTS_DIR = PROJECT_ROOT / "prompts"
DATA_DIR = PROJECT_ROOT / "data"
ORIGINAL_RESUME_DIR = DATA_DIR / "original"
JOB_POSTINGS_DIR = DATA_DIR / "job_postings"
OUTPUT_DIR = DATA_DIR / "tailored_versions"

# Detect which API credentials are available
print("🔍 Checking API credentials...")
print()

has_openai = bool(os.getenv('OPENAI_API_KEY'))
has_bedrock_token = bool(os.getenv('AWS_BEARER_TOKEN_BEDROCK'))
has_aws_creds = bool(os.getenv('AWS_ACCESS_KEY_ID'))

if has_openai:
    print("✅ OpenAI API key found")
    MODEL_PROVIDER = "openai"
    MODEL_ID = "gpt-4o"  # or gpt-4-turbo, gpt-4
elif has_bedrock_token:
    print("✅ AWS Bedrock bearer token found")
    MODEL_PROVIDER = "bedrock"
    MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
elif has_aws_creds:
    print("✅ AWS credentials found")
    MODEL_PROVIDER = "bedrock"
    MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
else:
    print("⚠️  Warning: No API credentials found!")
    print("Please set one of the following in .env file:")
    print("  - OPENAI_API_KEY (for OpenAI)")
    print("  - AWS_BEARER_TOKEN_BEDROCK (for Bedrock)")
    print("  - AWS_ACCESS_KEY_ID + AWS_SECRET_ACCESS_KEY (for AWS)")
    MODEL_PROVIDER = None
    MODEL_ID = None

print()
print(f"📡 Selected Provider: {MODEL_PROVIDER}")
print(f"🤖 Selected Model: {MODEL_ID}")

# Verify directories exist
print()
print(f"📁 Project directories:")
print(f"  Prompts: {PROMPTS_DIR.exists()} - {PROMPTS_DIR}")
print(f"  Data: {DATA_DIR.exists()} - {DATA_DIR}")
print(f"  Output: {OUTPUT_DIR.exists()} - {OUTPUT_DIR}")

## Load System Prompts

Load agent instructions from separate files for easy iteration.

In [ ]:
def load_prompt(filename: str) -> str:
    """Load a prompt from the prompts directory."""
    prompt_path = PROMPTS_DIR / filename
    if not prompt_path.exists():
        print(f"⚠️  Warning: {filename} not found. Using default prompt.")
        return ""
    
    with open(prompt_path, 'r', encoding='utf-8') as f:
        content = f.read()
    print(f"✅ Loaded {filename} ({len(content)} chars)")
    return content

# Load prompts
system_prompt = load_prompt("system_prompt.txt")
latex_rules = load_prompt("latex_rules.txt")

# Combine prompts
full_prompt = f"{system_prompt}\n\n{latex_rules}".strip()

print(f"\n📝 Full system prompt: {len(full_prompt)} characters")

## Custom Tools for Resume Tailoring

Define specialized tools for LaTeX resume processing.

In [ ]:
@tool
def read_file(filepath: str) -> str:
    """
    Read a file and return its contents.
    
    Args:
        filepath: Path to the file (relative to project root or absolute)
    
    Returns:
        The file contents as a string
    """
    try:
        path = Path(filepath)
        if not path.is_absolute():
            path = PROJECT_ROOT / filepath
        
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        return f"Error: File not found at {filepath}"
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def write_file(filepath: str, content: str) -> str:
    """
    Write content to a file.
    
    Args:
        filepath: Path to the file (relative to project root or absolute)
        content: Content to write
    
    Returns:
        Success message with file path
    """
    try:
        path = Path(filepath)
        if not path.is_absolute():
            path = PROJECT_ROOT / filepath
        
        # Create parent directories if needed
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path, 'w', encoding='utf-8') as f:
            f.write(content)
        
        return f"Successfully wrote {len(content)} characters to {path}"
    except Exception as e:
        return f"Error writing file: {str(e)}"


@tool
def validate_latex(latex_content: str) -> dict:
    """
    Validate LaTeX syntax by checking for common issues.
    
    Args:
        latex_content: The LaTeX content to validate
    
    Returns:
        Dictionary with validation results (is_valid, errors, warnings)
    """
    errors = []
    warnings = []
    
    # Check for balanced braces
    if latex_content.count('{') != latex_content.count('}'):
        errors.append("Unbalanced curly braces { }")
    
    # Check for balanced brackets
    if latex_content.count('[') != latex_content.count(']'):
        errors.append("Unbalanced square brackets [ ]")
    
    # Check for document structure
    if '\\documentclass' not in latex_content:
        warnings.append("No \\documentclass found")
    
    if '\\begin{document}' not in latex_content:
        errors.append("Missing \\begin{document}")
    
    if '\\end{document}' not in latex_content:
        errors.append("Missing \\end{document}")
    
    # Check for common LaTeX commands
    lines = latex_content.split('\n')
    for i, line in enumerate(lines, 1):
        # Check for unescaped special characters in regular text
        if '%' in line and '\\%' not in line:
            # This might be a comment, so it's just a warning
            pass
    
    is_valid = len(errors) == 0
    
    return {
        "is_valid": is_valid,
        "errors": errors,
        "warnings": warnings,
        "summary": f"{'✅ Valid' if is_valid else '❌ Invalid'} LaTeX ({len(errors)} errors, {len(warnings)} warnings)"
    }


@tool
def extract_keywords(text: str) -> list:
    """
    Extract important keywords from text (job posting or resume section).
    
    Args:
        text: Text to extract keywords from
    
    Returns:
        List of keywords (skills, technologies, requirements)
    """
    # Common technical keywords and skills
    import re
    
    # Simple keyword extraction (can be enhanced with NLP)
    keywords = set()
    
    # Common technical skills patterns
    patterns = [
        r'\b(Python|Java|JavaScript|TypeScript|C\+\+|Ruby|Go|Rust|Swift)\b',
        r'\b(AWS|Azure|GCP|Docker|Kubernetes|Jenkins)\b',
        r'\b(React|Angular|Vue|Node\.js|Django|Flask|Spring)\b',
        r'\b(SQL|PostgreSQL|MySQL|MongoDB|Redis)\b',
        r'\b(Git|CI/CD|Agile|Scrum|DevOps|REST|API)\b',
        r'\b(Machine Learning|AI|Data Science|Analytics)\b',
    ]
    
    for pattern in patterns:
        matches = re.finditer(pattern, text, re.IGNORECASE)
        for match in matches:
            keywords.add(match.group(1))
    
    return sorted(list(keywords))


print("✅ Custom tools defined:")
print("  - read_file()")
print("  - write_file()")
print("  - validate_latex()")
print("  - extract_keywords()")

## Create the Resume Tailor Agent

Initialize the agent with system prompts and custom tools.

In [ ]:
# Create agent with automatic provider detection
if MODEL_PROVIDER is None:
    print("❌ Cannot create agent: No API credentials found")
    print("Please configure API credentials in .env file")
else:
    # Create agent with detected provider
    resume_agent = Agent(
        model_provider=MODEL_PROVIDER,
        model_id=MODEL_ID,
        system_prompt=full_prompt if full_prompt else "You are a helpful resume tailoring assistant.",
        tools=[
            read_file,
            write_file,
            validate_latex,
            extract_keywords
        ]
    )

    print("✅ Resume Tailor Agent created!")
    print(f"   Provider: {MODEL_PROVIDER}")
    print(f"   Model: {MODEL_ID}")
    print(f"   Tools: {len(resume_agent.tools)} custom tools")
    print(f"   System prompt: {len(full_prompt) if full_prompt else 0} characters")
    print()
    print("💡 Tip: You can change the model by editing MODEL_PROVIDER and MODEL_ID in the configuration cell above")

---

## Usage Examples

Below are examples of how to use the resume tailor agent.

### Example 1: Test Agent Connection

In [ ]:
# Quick test
response = resume_agent("Hello! Can you help me tailor my resume?")
print(response)

### Example 2: Analyze a Job Posting

First, create a sample job posting file or use an existing one.

In [ ]:
# Create a sample job posting for testing
sample_job = """
Senior Software Engineer - Machine Learning

Requirements:
- 5+ years of experience in Python and machine learning
- Strong background in AWS cloud services
- Experience with PyTorch or TensorFlow
- Proficiency in SQL and data processing
- Excellent communication and teamwork skills

Preferred:
- PhD in Computer Science or related field
- Experience with MLOps and model deployment
- Knowledge of Docker and Kubernetes
"""

# Save sample job posting
job_file = JOB_POSTINGS_DIR / "sample_ml_job.txt"
job_file.parent.mkdir(parents=True, exist_ok=True)
with open(job_file, 'w') as f:
    f.write(sample_job)

print(f"✅ Sample job posting created at: {job_file}")

# Ask agent to analyze the job posting
response = resume_agent(
    f"Read the job posting from '{job_file.relative_to(PROJECT_ROOT)}' "
    "and extract the key requirements and skills."
)
print("\n" + "="*60)
print("AGENT ANALYSIS:")
print("="*60)
print(response)

### Example 3: Resume Tailoring Workflow

**Note**: You'll need to place your actual LaTeX resume in `data/original/resume.tex`

In [ ]:
# Reset the agent - creates a fresh instance with no conversation history
if MODEL_PROVIDER is None:
    print("❌ Cannot create agent: No API credentials found")
else:
    resume_agent = Agent(
        model_provider=MODEL_PROVIDER,
        model_id=MODEL_ID,
        system_prompt=full_prompt if full_prompt else "You are a helpful resume tailoring assistant.",
        tools=[
            read_file,
            write_file,
            validate_latex,
            extract_keywords
        ]
    )
    
    print("🔄 Agent reset successfully!")
    print(f"   Provider: {MODEL_PROVIDER}")
    print(f"   Model: {MODEL_ID}")
    print("   ✅ Fresh conversation - no previous context")
    print()
    print("💡 You can now start a new tailoring project with a clean slate")

### 🔄 Reset Agent (Optional)

Run this cell to reset the agent's conversation history without restarting the kernel. This clears all previous conversations while keeping your setup intact.

**When to use this:**
- Starting a new job posting/tailoring project
- Agent seems confused or has too much context
- You want a fresh conversation without re-running Examples 1-2

In [ ]:
# Define file paths
original_resume = "data/original/resume.tex"  # Your resume here
job_posting = "data/job_postings/sample_ml_job.txt"  # Job posting
output_file = "data/tailored_versions/resume_ml_engineer.tex"  # Output

# Instructions for the agent
tailoring_request = f"""
I need to tailor my resume for a specific job posting. Please:

1. Read my resume from: {original_resume}
2. Read the job posting from: {job_posting}
3. Analyze the job requirements and match them to my experience
4. Suggest specific improvements to tailor my resume
5. IMPORTANT: Preserve all LaTeX formatting and syntax

Do NOT generate the full tailored resume yet - just provide analysis and suggestions first.
"""

# Get initial analysis
print("Analyzing resume and job posting...\n")
analysis = resume_agent(tailoring_request)
print(analysis)

### Example 4: Iterative Refinement

After getting suggestions, you can iterate:

In [ ]:
# Continue the conversation for refinement
refinement_request = """
Based on your analysis, please:
1. Focus on highlighting my AWS and Python experience
2. Emphasize any machine learning projects
3. Ensure keywords match the job posting for ATS
4. Keep the resume to 1 page if possible

Show me the specific sections that should change.
"""

refinement = resume_agent(refinement_request)
print(refinement)

### Example 5: Generate Final Tailored Resume

In [ ]:
# Generate the final version
final_request = f"""
Please generate the final tailored resume based on our discussion.

Requirements:
1. Apply all the improvements we discussed
2. PRESERVE all LaTeX syntax and formatting
3. Validate the LaTeX before saving
4. Save the result to: {output_file}

After saving, confirm that the LaTeX is valid.
"""

result = resume_agent(final_request)
print(result)

### Example 6: Validate Output

In [ ]:
# Direct tool invocation for validation
if Path(output_file).exists():
    with open(output_file, 'r') as f:
        tailored_content = f.read()
    
    # Validate using tool directly
    validation = resume_agent.tool.validate_latex(latex_content=tailored_content)
    
    print("Validation Results:")
    print(f"  Valid: {validation['is_valid']}")
    print(f"  Errors: {len(validation['errors'])}")
    print(f"  Warnings: {len(validation['warnings'])}")
    
    if validation['errors']:
        print("\nErrors found:")
        for error in validation['errors']:
            print(f"  - {error}")
else:
    print(f"File not found: {output_file}")

---

## Helper Functions

Utility functions for common tasks.

In [ ]:
def quick_tailor(resume_path: str, job_path: str, output_path: str, instructions: str = ""):
    """
    Quick one-shot resume tailoring.
    
    Args:
        resume_path: Path to original resume
        job_path: Path to job posting
        output_path: Path for tailored resume
        instructions: Additional instructions for the agent
    """
    prompt = f"""
Tailor my resume for this job posting.

Resume: {resume_path}
Job Posting: {job_path}
Output: {output_path}

Steps:
1. Read both files
2. Analyze job requirements
3. Tailor resume content (preserve LaTeX formatting)
4. Validate LaTeX syntax
5. Save to output path

{instructions if instructions else ''}
"""
    
    response = resume_agent(prompt)
    return response


def batch_tailor(resume_path: str, job_folder: str, output_folder: str):
    """
    Tailor resume for multiple job postings.
    
    Args:
        resume_path: Path to original resume
        job_folder: Folder containing job posting files
        output_folder: Folder for tailored resumes
    """
    job_dir = Path(job_folder)
    output_dir = Path(output_folder)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    results = []
    
    for job_file in job_dir.glob("*.txt"):
        output_name = f"resume_{job_file.stem}.tex"
        output_path = output_dir / output_name
        
        print(f"\nTailoring for: {job_file.name}")
        result = quick_tailor(resume_path, str(job_file), str(output_path))
        results.append({"job": job_file.name, "output": output_name, "result": result})
    
    return results


print("✅ Helper functions defined:")
print("  - quick_tailor()")
print("  - batch_tailor()")

---

## Next Steps

1. **Add your resume**: Place your LaTeX resume in `data/original/resume.tex`
2. **Add job postings**: Save job postings as `.txt` files in `data/job_postings/`
3. **Run tailoring**: Use the examples above to tailor your resume
4. **Iterate**: Work with the agent to refine the output
5. **Validate**: Check LaTeX syntax before compiling
6. **Compile**: Use `pdflatex` or your LaTeX editor to generate PDF

### Tips for Best Results

- Start with analysis and suggestions before generating the full resume
- Be specific about what aspects to highlight
- Review the agent's suggestions before applying them
- Always validate LaTeX syntax
- Keep conversation context for iterative improvements
- Save different versions for different job types

### Troubleshooting

- **LaTeX errors**: Use `validate_latex()` tool to check syntax
- **Agent not following instructions**: Refine the system prompt in `prompts/system_prompt.txt`
- **Missing features**: Add custom tools as needed
- **Context lost**: Use conversation memory or save intermediate results